In [1]:
import os
import sys
import glob
import pandas as pd
import json

In [2]:
QC_item_list = ["ZMWs","ZMWs_filtered","FLNC Reads","FLNC_polyA Reads"]
sample_dict = {} 
'''
    "0009689008_CA":[,,,,,]
'''
result_path="/home/jovyan/work/10.data_CODA_ahslyy"


In [3]:
## parse the file of demultiplex.lima.summary
file_format="*.demultiplex.lima.summary"
lima_files=glob.glob(f"{result_path}/*_*/results/01.CCS_Trim/{file_format}")
for lima_file in lima_files:
    sample_name=os.path.basename(lima_file).split(".")[0]
    sample_dict[sample_name] = []
    lima_df = pd.read_table(lima_file,sep=":",names=["value"],index_col=0)
    ZMWs = lima_df.loc[lima_df.index.str.contains("ZMWs input"), "value"].values[0]
    ZMWs_above_thresholds = lima_df.loc[lima_df.index.str.contains("ZMWs above all thresholds"),"value"].values[0]
    sample_dict[sample_name].append(ZMWs)
    #sample_dict[sample_name].append(ZMWs_above_thresholds)

In [4]:
## parse the file of "flnc.filter_summary.report.json"
file_format="*flnc.filter_summary.report.json"
json_files = glob.glob(f"{result_path}/*_*/results/01.CCS_Trim/{file_format}")
for json_file in json_files:
    sample_name = os.path.basename(json_file).split(".")[0]
    if sample_name not in sample_dict.keys():
        sample_dict[sample_name] = []
    with open(json_file,"r") as json_f:
        json_data = json.load(json_f)
        print()
        sample_dict[sample_name].append(json_data["attributes"][1]["value"])  ## append "Full-Length Reads”
        sample_dict[sample_name].append(json_data["attributes"][2]["value"]) ## append "Full-Length Non-Chimeric Reads"
        sample_dict[sample_name].append(json_data["attributes"][3]["value"])  ## append "Full-Length Non-Chimeric Reads with Poly-A Tail"

In [ ]:
## parse the file of "flnc.filter_summary.report.json"
file_format="*flnc.filter_summary.report.json"
json_files = glob.glob(f"{result_path}/*_*/results/01.CCS_Trim/{file_format}")
for json_file in json_files:
    sample_name = os.path.basename(json_file).split(".")[0]
    if sample_name not in sample_dict.keys():
        sample_dict[sample_name] = []

In [14]:
df = pd.DataFrame(sample_dict,index=QC_item_list)
df.to_csv(os.path.join(result_path,"QC_summary.csv"))